In [2]:
import polars as pl
#https://catalog.data.gov/dataset/electric-vehicle-population-data
lazy_car_data= pl.scan_csv('/Volumes/developer/Pycharm/data/Electric_Vehicle_Population_Data.csv')

In [4]:
lazy_car_data.collect_schema()

Schema([('VIN (1-10)', String),
        ('County', String),
        ('City', String),
        ('State', String),
        ('Postal Code', Int64),
        ('Model Year', Int64),
        ('Make', String),
        ('Model', String),
        ('Electric Vehicle Type', String),
        ('Clean Alternative Fuel Vehicle (CAFV) Eligibility', String),
        ('Electric Range', Int64),
        ('Base MSRP', Int64),
        ('Legislative District', Int64),
        ('DOL Vehicle ID', Int64),
        ('Vehicle Location', String),
        ('Electric Utility', String),
        ('2020 Census Tract', Int64)])

In [11]:
lazy_car_query = (
     lazy_car_data
     .filter((pl.col("Model Year") >= 2018))
     .filter(
         pl.col("Electric Vehicle Type") == "Battery Electric Vehicle (BEV)"
     )
     .group_by(["State", "Make"])
     .agg(
         pl.mean("Electric Range").alias("Average Electric Range"),
         pl.min("Model Year").alias("Oldest Model Year"),
        pl.len().alias("Number of Cars"),
    )
     .filter(pl.col("Average Electric Range") > 0)
   .filter(pl.col("Number of Cars") > 5)
     .sort(pl.col("Number of Cars"), descending=True)
)

In [12]:
lazy_car_query.collect()

State,Make,Average Electric Range,Oldest Model Year,Number of Cars
str,str,f64,i64,u32
"""WA""","""TESLA""",60.016801,2018,82021
"""WA""","""CHEVROLET""",94.878229,2018,8245
"""WA""","""NISSAN""",72.131715,2018,6962
"""WA""","""FORD""",0.121988,2018,6558
"""WA""","""KIA""",38.988855,2018,5563
…,…,…,…,…
"""NC""","""TESLA""",19.545455,2018,11
"""MD""","""TESLA""",46.0,2018,11
"""FL""","""TESLA""",104.428571,2019,7


KeyboardInterrupt: 